In [1]:
# Copyright 2018 Anar Amirli
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train/Test Set Construction
In this document, we construct training and test sets from already computed feature sets. The sets are computed according to the layout:

data<br>
├── train_features<br>
├── test_features<br>

In [2]:
from os import listdir
from os.path import join
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from collections import Counter
from random import shuffle
import json
import re

# our common utility functions that are used in different notebooks
from utils import *

pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [6]:
def describe_dataset(dataset):
    _, col = dataset.shape
        
    heading = 'Pitch index counts'
    print(heading + '\n' + '-'*len(heading))
    for key, val in sorted(Counter(dataset.values[:, col-4]).items()):
        print('{}\t: {}'.format(int(key), val))

## Pitch IDS

In [4]:
if __name__ == '__main__':

    match_id = 60565
    try:
        features_df = pd.read_csv('../data/general/feature-set/match{}_features_2.csv'.format(match_id))
    except FileNotFoundError:
        pass

    ball_idxs = np.sort(features_df['pitch_index'].unique())
    print('Ball  index: {}'.format(ball_idxs))

    print('Size: {}'.format(len(ball_idxs)))

Ball  index: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
Size: 10


Null index(0) represents all the event categories, apart from the ones we are interested in, that occur when the game stops.

## Dataset Construction

In this section, we construct a combined shuffled dataset for train and set sets, coming from feature data.


### Parameters

When obtaining feature data for a given index, set we should reduce the number of frames of the null ball events in our data set. But cosidering the fact that they are most common, we should keep their numbers close to the number of other most occurred ball location indexes.

* $-$: No-labaled positions intervals would lead to greater label noise.

### Construction
Here we construct the combined dataset from all match data we have in the given data directories.

In [7]:
    # file based constants
    features_dir     = '../data/general/feature-set/'
    features_regex   = re.compile(r'match\d+_features_2.csv')

We speed up the computation by using all the CPU cores via multiprocessing module.

In [8]:
    pool = multiprocessing.Pool()
    df = pd.DataFrame()

    
    features_csv_files = [f for f in listdir(features_dir) if features_regex.match(f)]
    shuffle(features_csv_files)
    
    print('# all matches: {}'.format(len(features_csv_files)))
    features_all_csv = features_csv_files[0:len(features_csv_files)]
    df_all_data = pd.concat(pool.map(construct_data_set, features_all_csv))

    print('All-data-set shape: {}'.format(df_all_data.shape))
    
    describe_dataset(df_all_data)

# all matches: 298
All-data-set shape: (679382, 232)
Pitch index counts
------------------
0	: 209737
1	: 19797
2	: 47201
3	: 18229
4	: 57806
5	: 169396
6	: 58463
7	: 21156
8	: 52785
9	: 24812


## Exporitng 

In [9]:
    df_all_data.to_csv('../data/general/{dataset}/data_{dataset}.csv'.format(dataset='all'), index=False)

In [10]:
    df_all_data[1:10]

,# refX,refY,refSpeed,refDirectX,refDirectY,teamsAvgSpped,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,...,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index,xpos,ypos,ball_owner
1,32.15625,44.031250,3.759766,-1.0,1.0,3.453125,37.75000,33.718750,35.31250,31.90625,...,0.000000,0.0000,0.00000,31.437500,30.65625,5.710938,5.0,59.46875,24.750000,1.0
2,63.56250,37.062500,0.919922,-1.0,-1.0,1.228516,53.71875,38.500000,60.90625,38.50000,...,40.906250,0.0000,0.00000,0.000000,0.00000,0.000000,9.0,91.25000,53.281250,1.0
3,51.59375,41.687500,1.049805,-1.0,1.0,1.544922,48.68750,40.906250,47.62500,38.03125,...,49.406250,0.0000,0.00000,0.000000,0.00000,0.000000,0.0,0.00000,0.000000,0.0
4,42.25000,29.265625,1.530273,1.0,-1.0,2.154297,48.84375,28.421875,37.43750,25.43750,...,55.343750,0.0000,0.00000,0.000000,0.00000,0.000000,4.0,59.62500,3.480469,1.0
5,41.65625,42.343750,2.320312,1.0,-1.0,3.115234,44.06250,48.156250,47.00000,41.43750,...,0.000000,48.6875,64.37500,0.000000,0.00000,0.000000,5.0,72.62500,23.703125,1.0
6,48.68750,40.968750,5.328125,-1.0,1.0,4.769531,44.46875,38.687500,42.15625,37.81250,...,0.000000,0.0000,0.00000,31.171875,29.56250,5.089844,5.0,43.03125,46.781250,1.0
7,41.06250,31.437500,2.320312,1.0,1.0,2.263672,45.46875,26.500000,43.75000,36.12500,...,17.984375,0.0000,0.00000,0.000000,0.00000,0.000000,5.0,67.81250,31.296875,1.0
8,60.87500,38.250000,2.720703,-1.0,1.0,1.811523,49.59375,33.687500,49.59375,33.68750,...,0.000000,46.0625,40.03125,0.000000,0.00000,0.000000,8.0,79.56250,27.750000,1.0
9,48.15625,51.593750,2.380859,1.0,-1.0,1.003906,43.06250,47.937500,39.00000,48.21875,...,63.156250,0.0000,0.00000,0.000000,0.00000,0.000000,0.0,0.00000,0.000000,0.0
